In [ ]:
import copy
import os

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from transformers import AdamW, AutoConfig, AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup

In [5]:
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-large-4096')
#tokenizer = T5Tokenizer.from_pretrained("t5-large'
path = 'feedback-prize-2021'
df = pd.read_csv(os.path.join(path,'train.csv'))
train_ids = df['id'].unique()

def _prepare_training_data_helper(path, tokenizer, df, train_ids):
    training_samples = []
    for idx in tqdm(train_ids):
        filename = os.path.join(path, "train", idx + ".txt")
        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=False,
            return_offsets_mapping=True,
        )
        input_ids = encoded_text["input_ids"]
        input_labels = copy.deepcopy(input_ids)
        offset_mapping = encoded_text["offset_mapping"]

        for k in range(len(input_labels)):
            input_labels[k] = "O"

        sample = {
            "id": idx,
            "input_ids": input_ids,
            "text": text,
            "offset_mapping": offset_mapping,
        }

        temp_df = df[df["id"] == idx]
        # each row of the df is going to be one sentence with one type
        for _, row in temp_df.iterrows():
            # id is just one document
            text_labels = [0] * len(text)
            discourse_start = int(row["discourse_start"])
            discourse_end = int(row["discourse_end"])
            prediction_label = row["discourse_type"]
            text_labels[discourse_start:discourse_end] = [1] * (discourse_end - discourse_start)
            target_idx = []
            # iterating over the offset mapping for the encoded text (tokenized text) so you are basically iterating over every single word (token)
            for map_idx, (offset1, offset2) in enumerate(encoded_text["offset_mapping"]):
                if sum(text_labels[offset1:offset2]) > 0:
                    if len(text[offset1:offset2].split()) > 0:
                        target_idx.append(map_idx)

            targets_start = target_idx[0]
            targets_end = target_idx[-1]
            pred_start = "B-" + prediction_label
            pred_end = "I-" + prediction_label
            input_labels[targets_start] = pred_start
            input_labels[targets_start + 1 : targets_end + 1] = [pred_end] * (targets_end - targets_start)

        sample["input_ids"] = input_ids
        sample["input_labels"] = input_labels
        training_samples.append(sample)
    return training_samples

training_sample = _prepare_training_data_helper(path, tokenizer, df, train_ids[:100]) 

100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 87.47it/s]


In [6]:
print(len(training_sample[0]['input_ids']))
print(len(training_sample[0]['text'].split()))
print(training_sample[0])

444
379
{'id': '423A1CA112E2', 'input_ids': [48083, 50118, 50118, 39631, 5868, 452, 32, 460, 15, 49, 1028, 4, 252, 32, 460, 15, 49, 1028, 55, 87, 195, 722, 10, 183, 117, 912, 479, 3684, 51, 109, 16, 2788, 124, 8, 556, 8, 95, 33, 333, 732, 2923, 15, 592, 433, 4, 252, 190, 109, 24, 150, 1428, 4, 252, 32, 103, 269, 1099, 4914, 77, 2682, 2594, 77, 24, 606, 7, 10, 1028, 4, 993, 1402, 911, 11, 5, 315, 532, 2020, 4247, 31, 1380, 5351, 95, 142, 9, 24, 4, 50118, 50118, 1779, 82, 33, 4247, 6, 51, 216, 59, 1402, 3798, 14, 51, 33, 479, 46036, 101, 622, 599, 1838, 8, 11477, 4, 407, 101, 114, 10, 1441, 3136, 409, 8, 47, 236, 7, 28, 11, 1511, 47, 64, 202, 28, 11, 1511, 30, 6016, 3424, 50, 2788, 3731, 4, 1806, 460, 33, 430, 1319, 141, 7, 8469, 19, 10, 1028, 4, 4129, 6909, 33, 1714, 528, 7, 84, 2706, 4, 50118, 50118, 34002, 6645, 16, 65, 9, 5, 169, 141, 7, 120, 198, 4, 1806, 460, 28, 15, 49, 4247, 150, 608, 24, 4, 6834, 64, 1303, 1473, 35677, 4, 280, 18, 596, 89, 18, 10, 631, 14, 18, 373, 117, 19943, 1

In [ ]:
df['discourse_type'].unique()